In [5]:
# I have two new pdfs to check
# i also need to translate my current data to dutch!
# now trying to get some QnA from documents ... bon it's very bad
# maybe do something that makes calls to GPT of EY? ...

In [3]:
import pandas as pd

context = pd.read_json(
    "/Users/naimsassine/Desktop/DSAI/Belgian-law-finetuning/Data/jsonl_files/all_text.jsonl",
    lines=True,
)

In [9]:
import re

# Path to your text file
file_path = "/Users/naimsassine/Desktop/DSAI/keys"

# Read the content of the file
with open(file_path, "r") as file:
    content = file.read()

# Regular expressions to match the key, base, and version
api_key_pattern = r'openai\.api_key\s*=\s*"([^"]+)"'
api_base_pattern = r'openai\.api_base\s*=\s*"([^"]+)"'
api_version_pattern = r'openai\.api_version\s*=\s*"([^"]+)"'

# Extract the values using the regular expressions
api_key = re.search(api_key_pattern, content).group(1)
api_base = re.search(api_base_pattern, content).group(1)
api_version = re.search(api_version_pattern, content).group(1)

In [11]:
import openai
import pandas as pd

# Set your OpenAI API key
openai.api_key = "your_openai_api_key"

# Assuming your dataframe is named 'context' and has a column 'text'
context = pd.DataFrame(
    {
        "text": [
            "Article 1: La Belgique est un État fédéral qui se compose des communautés et des régions.",
            "Article 2: La Belgique comprend trois communautés : la Communauté flamande, la Communauté française et la Communauté germanophone.",
            # Add more lines as needed
        ]
    }
)


def generate_questions_and_answers(text):
    prompt = (
        f'Pour le texte suivant de la constitution belge :\n\n"{text}"\n\n'
        "Génère 5 questions et leurs réponses en français."
    )
    response = openai.Completion.create(
        engine="davinci",
        prompt=prompt,
        max_tokens=200,
        n=1,
        stop=None,
        temperature=0.7,
    )
    return response.choices[0].text.strip()


# Create a new dataframe to store the results
results = []

for index, row in context.iterrows():
    text = row["text"]
    qa_pairs = generate_questions_and_answers(text)
    results.append({"text": text, "qa_pairs": qa_pairs})

# Convert results to a new dataframe
qa_df = pd.DataFrame(results)

# Display the results
print(qa_df)

,text
0,Belgium is a federal State composed of Communi...
1,Belgium comprises three Communities: the Fl em...
2,Belgium comprises three Regions: the Flemish R...
3,Belgium comprises four linguistic regions: the...
4,The Flemish Region comprises the following pro...
...,...
1765,"art. 22, 051; inwerkingtreding : 08-08-2005 (w..."
1766,"art. 4, 048; inwerkingtreding : 01-04-2005> (2..."
1767,art. 563bis. [1 met geldboete van vijftien eur...
1768,"art. 22, 051; inwerkingtreding : 08-08-2005 (w..."
